# 0. What is visit history?
* Ｇoal: 分析Chrome瀏覽器的使用歷程檔
* What-is: 瀏覽器的log（即所謂的使用歷程檔）是什麼？上面有什麼資訊？請依照以以下連結的指示，開啟Chrome瀏覽器，找到右上角的工具選項，打開「記錄」。 http://www.googlechrometutorial.com/google-chrome-general-settings/Google-chrome-web-browser-history.html
* Reference
    * [How Google Chrome stores web history](http://lowmanio.co.uk/blog/entries/how-google-chrome-stores-web-history/)
    * http://superuser.com/questions/627253/storing-browser-history-for-personal-analytics
    * http://www.v2ex.com/t/60255

#1. Copy chrome history out
* 如何獲取Chrome History檔案？
* 「access chrome history data」，之後應可找到下列網址 http://stackoverflow.com/questions/2562092/how-to-access-google-chrome-browser-history-programmatically-on-local-machine
解釋】從上面鏈結可獲知Chrome History以sqlite資料庫的檔案形態放在使用者個人電腦端（並非在雲端）

## Windows
* 如果是Windows的話是放在「C:\Documents and Settings\Local Settings\Application Data\Google\Chrome\User Data\Default」檔名為History。
* 該檔案可能是隱藏檔，所以你必須自「資料夾選項」中開啟可看見隱藏檔的選項（以下資訊參考http://www.chromium.org/user-experience/user-data-directory）。
* __Win8__:  C:\Documents and Settings\Local Settings\Application Data\Google\Chrome\User Data\Default
* __Win 7__: C:\Users\[USERNAME]\AppData\Local\Google\Chrome\User Data\Default
* __WinXP__: C:\Documents and Settings\[USERNAME]\Local Settings\Application Data\Google\Chrome\User Data\Default

##  Mac
* 若使用Mac電腦的話，History檔可能會放置在以下兩個位址：
    * /Library（資源庫）/Application Support/Google/Chrome/Default/History
    * /Users/[Your name]/Library（資源庫）/Application Support/Google/Chrome/Default/History
* Mac的History檔為隱藏檔，需要用以下的方法取消隱藏
    * 如果該檔案被隱藏無法搜尋的話，請在spotlight搜尋terminor，會開啟如下視窗（方法來源：view hidden folders on your mac using terminal０）
        * 輸入defaults write com.apple.finder AppleShowAllFiles TRUE 
        * 輸入killall Finder即可找到該資料夾
        * 之後請再輸入defaults write com.apple.finder AppleShowAllFiles FALSE 
        * 再輸入killall Finder即可恢復原狀。
    * <font color = red>或者用這方法：打開Finder後，按住Option，點選上方「前往」中的「前往資料夾」，然後直接貼上上述兩個網址之一，注意要把（資源庫）拿掉，並且把你的使用者名稱貼上。例如：rawdata/Users/jirlong/Library/Application Support/Google/Chrome/Default/History</font>

#2. Browse the History file
* Chrome history是sqlite檔而無法被一般的程式（例如NotePad）開啟。 那麼，何謂sqlite檔案？他和一般資料庫檔案（例如.mdb）有何不同？
* 無論是Firefox或Chrome history均以sqlite的資料庫格式儲存成為一個檔案，所以現在需要用一個sqlite browser來開啟所下載的history檔。
* 請上網搜尋"sqlite reader mac"或"sqlite reader windows"（Mac或Windows視你所用的系統而定，關鍵字reader可改為browser），下載並安裝。http://sourceforge.net/projects/sqlitebrowser/
* 下載該工具並且打開你所下載的History檔案。觀查History的內容，並告訴我裡面記錄了哪些資料，又以何種形式來記錄？

##2.1 Brief introduction to chrome history
* 安裝完畢後用其打開你所下載的History檔案，應可見到如下的界面。共分為三個主要部分，分別為：
    * Database Structure：用以觀看整個資料庫的綱要或欄位
    * Browse Data：用以瀏覽資料庫內存的資料<--我們最主要要看這個資料。
    * Execute SQL：允許你用這個界面操作SQL指令以查詢
    * <img src="Figures/sqlbrowser.png" style="width: 50%; height: 50%"/>
* 點選Browse Data後應可見到以下界面，開啟Table的下拉式選單，一共可發現以下資料庫的資料表：
    * meta
    * downloads：下載的檔案
    * download_url_chains
    * urls：曾經拜訪過的urls的ID與網址對照表
    * visits：主要的瀏覽歷程資料表，其中需要以ID查詢瀏覽時所拜訪的urls（查詢前項urls資料表）
    * visit_source
    * keyword_search_terms：查詢關鍵字
    * segments
    * segment_usage
* 請開啟urls資料表並開啟「檔案>export」的選項將該資料表另存為csv（comma-separated value）檔。再開啟visits資料表，並用上述的方法另存為csv檔。只要儲存為csv檔後，不僅易於程式處理，而且匯入Excel也會十分方便。

##2.2 Tables of chrome history
* History Table – urls. urls資料表主要記錄了你所曾經拜訪過的所有的網頁網址，每一筆記錄就是一筆唯一的網址，如果你曾經拜訪過某網址兩次，他也只會有一筆記錄，它記錄了以下欄位：
    * id：該網址的id，用以索引
    * url：該網址本身
    * title：該網頁的title（可能會從缺）
    * visit_count：使用者在這段時間中一共拜訪該網頁幾次
    * typed_count
    * last_visit_time：使用者最後一次拜訪該網頁的時間（你在看網頁的時候，他有時候會告訴你「您最近一次拜訪時間為...」就是看這個欄位）
    * hidden：是否為隱藏的記錄
    * favicon_id

## 2.3 SQL queries to retrieve data
* 通常類似的SQLite Database Browser都會提供輸入資料庫查詢命令的功能，點選Execute SQL後即可看到輸入SQL查詢命令的欄位。例如我們希望查詢urls這個表中，究竟有哪些網址，並且按照該網址的拜訪次數來排序的話，可以輸入「select visit_count, url from urls order by visit_count」測試之。
    * 查詢一：我希望從urls這個表格中把visit_count和url這兩個欄位的資料給選擇出來，並且按照url這個欄位進行排序。
            select visit_count, url from urls order by url 
    * 查詢二：我希望從visits這個表格中把所有的visit給找出來，並且找出url, visit_time(拜訪時間), from_visit(從哪個URL跳過來), transition(怎麼跳過來), visit_duration(拜訪多久)。且所有的結果照著id來排序。
            select url,visit_time, from_visit, transition, visit_duration from visits order by id;
    * 查詢三：我希望整合visits和urls這兩個表的資料，依據是visits表的url等於urls表的id。同時找出visits表的拜訪時間、拜訪時間長，以及這個urls表的url（因為visits表的url存的是一個編碼）。
            SELECT visits.id, visits.visit_time, visits.visit_duration, urls.url FROM visits INNER JOIN urls ON visits.url=urls.id;
* 如果我今天想要查詢資料表visits中的拜訪時間、拜訪的持續時間、拜訪的網址那要怎麼做？此時你若打開資料表visits會發現拜訪網址欄位為url，其內容是一個編號，這個id必須參考對照資料表urls的id和url欄位。因此必須要輸入以下指令：
        SELECT visits.id, visits.visit_time, visits.visit_duration, urls.url
        FROM visits
        INNER JOIN urls
        ON visits.url=urls.id;



#3. Read Chrome history by python
##3.1  import sqlite3
* 為了讀SQLite檔案，必須import sqlite3函式庫，該函式庫為Python安裝時所內建的函式庫，只需import不用另外安裝。
* 把下載下來的History檔案放在同資料中如下圖即可讀取。之後參見https://docs.python.org/2/library/sqlite3.html，
* 該文件的內容說明了sqlite3這個函式庫可以讓使用者建立table、寫入資料並讀取資料庫。

In [6]:
import sqlite3
import urlparse
import datetime

##3.2 Connect to history sqlite file

In [7]:
conn=sqlite3.connect("../../log_ChromeHistory/History")
cu=conn.cursor()

##3.3 執行sql查詢（Send a sql query to cu）
* w3school SQL Introduction http://www.w3schools.com/sql/sql_intro.asp
* An example: 從（from）urls這個表中，依序選出（select）visit_count與url這兩個欄位，並且照url排序（order by）
        select visit_count, url from urls order by url;

In [1]:
sql = "select visit_count, url from urls order by url;"
cu.execute(sql)
res=cu.fetchall()
print "[COMMAND(num_of_res=%d)]:%s"%(len(res), sql)
for r in sorted(res, reverse=True)[:10]:
    print "%s\t%s"%(r[0], r[1])

NameError: name 'cu' is not defined

## 3.4 網址的處理 Combine urls according to their hosts
* 注意看下面query後的結果，顯然如果是mail.google.com開頭的網址，應該都算進mail.google.com就好，不要有那麼多個不一樣的網址。
        874	http://mail.google.com/
        654	https://www.facebook.com/
        500	http://www.facebook.com/
        429	https://mail.google.com/
        407	https://mail.google.com/mail/
        403	https://mail.google.com/mail/u/0/
        292	https://mail.google.com/mail/u/0/#label/%5Bntnulib%5D
        270	http://comic.sfacg.com/
        255	http://www.yahoo.com.tw/
        219	http://tw.yahoo.com/
* 所以我現在希望能夠把前面開頭是mail.google.com的都算成是拜訪同一個網址，並且累計總共拜訪幾次。
* 相當於要建一個dictionary 把網址前段mapping到出現次數的加總。
* 而現在的問題是要怎麼把網址斷開？

### 3.4.1 string operations
* 各位是否還記得我們曾經看過一個程式如下，他用sentences.split()把所有的字通通斷開了。
    1. 把所有的奇怪的字元刪除，包含
        >!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
            sentences = sentences.translate(None, string.punctuation)
    2. 把所有的字轉小寫
            sentences = sentences.lower()
    3. 把要輸入的字串依照空白斷開
            wordlist = sentences.split()
    4. 用for-each讀取wordlist中的每一個word，看他出現幾次。

In [5]:
import string
sentences = 'All men have stars, but they are not the same things for different people. For some, who are travelers, the stars are guides. For others they are no more than little lights in the sky. For others, who are scholars, they are problems... But all these stars are silent. You-You alone will have stars as no one else has them... In one of the stars I shall be living. In one of them I shall be laughing. And so it will be as if all the stars will be laughing when you look at the sky at night..You, only you, will have stars that can laugh! And when your sorrow is comforted (time soothes all sorrows) you will be content that you have known me... You will always be my friend. You will want to laugh with me. And you will sometimes open your window, so, for that pleasure... It will be as if, in place of the stars, I had given you a great number of little bells that knew how to laugh'
sentences = sentences.translate(None, string.punctuation)
sentences = sentences.lower()
wdict = {}
for word in sentences.split():
    if not wdict.has_key(word):
        wdict[word] = 0
    wdict[word] += 1
for f, q in sorted(((freq, word) for word, freq in wdict.items()), reverse=True)[:10]:
    print f, q
plt.hist(wdict.values(), facecolor='green', alpha=0.5)# hist:histogram

9 will
8 you
8 stars
7 the
7 be
7 are
5 for
4 that
4 of
4 in


NameError: name 'plt' is not defined

### 3.4.2 split the url by "/"

In [4]:
urlstr = 'https://mail.google.com/mail/u/0/#label/%5Bntnulib%5D'
# print urlstr.split('/')
urlseq = urlstr.split('/')
print urlseq[2]

mail.google.com


In [9]:
## combine urls according to the 1st segment of url

url_dict = {}
for item in res:
    urlsegs = item[1].split('/')
    if len(urlsegs) >= 2:
        url_dict.setdefault(urlsegs[2], 0)
        url_dict[urlsegs[2]] += item[0]
res_list = []
for k, v in url_dict.items():
    res_list.append([v, k])
for r in sorted(res_list, reverse=True)[:20]:
    print r

#     try:
#         url_dict.setdefault(urlseg[2], 0)
#         url_dict[urlseg[2]] += item[0]
#     except:
#         pass
#     res_list = url_dict.items()
#     res_list.sort(key=lambda s:s[1], reverse=True)
#     return res_list

[4674, u'mail.google.com']
[3166, u'comic.sfacg.com']
[3076, u'www.google.com']
[1731, u'www.facebook.com']
[1295, u'www.youtube.com']
[907, u'moodle.ntnu.edu.tw']
[883, u'www.google.com.tw']
[764, u'tw.news.yahoo.com']
[570, u'coldpic.sfacg.com']
[551, u'docs.google.com']
[386, u'jlis.glis.ntnu.edu.tw']
[362, u'www.books.com.tw']
[301, u'plus.google.com']
[255, u'www.yahoo.com.tw']
[237, u'remarks.sinaapp.com']
[236, u'translate.google.com']
[219, u'tw.yahoo.com']
[203, u'local-ntp']
[202, u'drive.google.com']
[199, u'www.mobile01.com']


##3.5 Convert the block of database query to a function
* 因為下面這兩行常常會用到，不如把它轉成一個「功能性的區塊」，就是function。
        cu.execute(sql)
        res=cu.fetchall()
* y = func(a, b) = sqrt(axa + bxb)
    * y稱為傳回值 return value
    * a, b為傳入函式的值。
    * func(a, b) = sqrt(axa + bxb) 稱為func函式的定義。


In [13]:
def query(sql):
    cu.execute(sql)
    res=cu.fetchall()
    return res

In [ ]:
# res = query("select url,visit_time, from_visit, transition, visit_duration from visits order by id ;") 
# res = query("SELECT visits.id, visits.visit_time, visits.visit_duration, urls.url FROM visits INNER JOIN urls ON visits.url=urls.id;")

In [14]:
# def top_n_url2(cu, n=None, sql="select url,visit_count from urls order by url ;"):
#     res = query(cu, sql)
#     uniq_res = {}
#     for item in res:
# #         urlseg = item[0].split('/')
# #         print urlseg[2]
#         urlhost = urlparse.urlparse(item[0]).netloc   # for getting host from url
#         uniq_res.setdefault(urlhost, 0)
#         uniq_res[urlhost] += item[1]
#     res_list = uniq_res.items()
#     res_list.sort(key=lambda x:x[1],reverse=True)
#     for r in res_list[:10]:
#         print r
#     return res_list[:n]
# res = top_n_url2(cu, 10)

##3.6 Query the visits table by the following sql
* 用sql browser觀察visits這個表，發現表中的url欄位儲存的不是真正的網址，而只有儲存網址的id。
* 而這個id，要對應到urls這個資料表的id欄位，才可以查出他所對應到的網址是什麼（urls中的url欄位）。所以要寫一道sql的查詢指令，"INNER JOIN"兩個表（你在這邊不需要知道INNER JOIN是什麼，只要你知道下面這道指令的意思就好了）。
        SELECT urls.url, visits.visit_time, visits.visit_duration, visits.id, visits.from_visit, visits.transition, visits.segment_id, urls.id FROM visits INNER JOIN urls ON visits.url=urls.id;
* 如果你想更瞭解SQL指令，可以參閱w3school上的說明。w3school SQL Introduction http://www.w3schools.com/sql/sql_intro.asp。

In [18]:
res=query("SELECT urls.url, visits.visit_time, visits.visit_duration, visits.id, visits.from_visit, visits.transition, visits.segment_id, urls.id FROM visits INNER JOIN urls ON visits.url=urls.id;")
print res[:2]

[(u'https://www.google.com.tw/search?q=ma&oq=ma&aqs=chrome..69i57j69i65j0j5j0.361j0&sourceid=chrome&ie=UTF-8', 13024375992405427, 4763264, 1, 0, 838860805, 0, 261), (u'http://mail.google.com/', 13024375995839026, 0, 2, 0, 268435457, 1, 262)]


##3.7 Convert timestamp to readable string
* 針對這樣的結果，看似有一些手續要處理，第一個，取出網址的前段，第二個，轉換時間成為python讀得懂的格式，不然根本看不懂。
* 這個看似整數的時間是網路處理時間的方法，為了讓python讀得懂這串數字是什麼，可以使用人家已經寫好的時間函式，把整數timestamp轉換為python datetime的方法可查詢"convert timestamp to datetime python"
    * 獲得以下結果http://stackoverflow.com/questions/9744775/how-to-convert-integer-timestamp-to-python-datetime
            >>> import datetime
            >>> your_timestamp = 1331856000000
            >>> date = datetime.datetime.fromtimestamp(your_timestamp / 1e3)
* 因為該整數是以microsecond來累計，所以要除以1000，標準的做法是除以1e3。
* duration也要轉，

In [24]:
epoch_start = datetime.datetime(1601,1,1) + datetime.timedelta(hours = 8)
for r in res[:10]:
    # retrieve the first segment of url
    urlhost = urlparse.urlparse(r[0]).netloc
    # convert integer time stamp to datetime
    print r[1]
    dt = datetime.timedelta(microseconds = r[1]) ## 1e3 means 1000
    duration = datetime.timedelta(microseconds = r[2])
    print urlhost, epoch_start+dt, duration


13024375992405427
www.google.com.tw 2013-09-23 10:13:12.405427 0:00:04.763264
13024375995839026
mail.google.com 2013-09-23 10:13:15.839026 0:00:00
13024375995839026
mail.google.com 2013-09-23 10:13:15.839026 0:00:00
13024375996831155
mail.google.com 2013-09-23 10:13:16.831155 0:00:00
13024375996831155
accounts.google.com 2013-09-23 10:13:16.831155 0:00:00
13024376003875352
accounts.google.com 2013-09-23 10:13:23.875352 0:01:58.716769
13024376004883494
mail.google.com 2013-09-23 10:13:24.883494 0:00:00
13024376004883494
mail.google.com 2013-09-23 10:13:24.883494 0:00:00
13024376004883494
mail.google.com 2013-09-23 10:13:24.883494 0:00:00
13024376006908314
mail.google.com 2013-09-23 10:13:26.908314 0:00:00


##3.8 getTime() convert timestamp to python datatime

In [ ]:
def getTime(timestamp, pivot="time"):
    newtime = str(timestamp)
    stripped1 = newtime.strip(' (),L')
    ms = int(stripped1)
    delta = datetime.timedelta(microseconds = ms)
    epoch_start = datetime.datetime(1601,1,1) + datetime.timedelta(hours = 8)

    if pivot=="duration":
        return delta
    else:
        return epoch_start + delta

In [ ]:
# def loaddata(cu):
#     print ".......LOADING DATA"
#     segment_dict, query_dict = {}, {}
#     res = query(cu, "select id, name, url_id from segments;")
#     segment_dict[0]="UNKNOWN"
#     for r in res:
#         segment_dict[r[0]] = r[1]
#     res = query(cu, "select url_id, lower_term from keyword_search_terms;")
#     for r in res:
#         query_dict[r[0]] = r[1]
#     print segment_dict
#     return query_dict, segment_dict
# query_dict, segment_dict = loaddata(cu)

In [ ]:
def get_transition(transition):
    code = 0xff
    transition = code & transition

    flag=""
    if transition==0:
        flag = "CLICK_LINK"     ##"LINK: User reached page by clicking a link on another page"
    elif transition==1:
        flag = "TYPE_URL"       ##"TYPED: User typed page in URL bar"
    elif transition==2:
        flag = "AUTO_BOOKMARK"  ##"AUTO BOOKMARK: User got to this page through a suggestion in the UI"
    elif transition==3:
        flag = "AUTO_SUBFRAME"  ##"AUTO SUBFRAME: Content automatically loaded in a non-toplevel frame"
    elif transition==4:
        flag = "MANU_SUBFRAME"  ##"MANUAL SUBFRAME: Subframe navigation explicitly requested by the user"
    elif transition==5:
        flag = "GENERATED"      ##"GENERATED: User typed page in the URL bar and selected an entry that did not look like a URL"
    elif transition==6:
        flag = "START PAGE"     ##"START PAGE: Page was specified in the command line or is the start page"
    elif transition==7:
        flag = "FORM SUBMIT"    ##"FORM SUBMIT: User filled out values in a form and submitted it"
    elif transition==8:
        flag = "RELOAD"         ##"RELOAD: User reloaded the page"
    elif transition==9:
        flag = "KEYWORD"        ##"KEYWORD: URL generated from a replaceable keyword other than the default search provider"
    elif transition==10:
        flag = "KEYWORD GENERATED"  ##"KEYWORD GENERATED: Visit was generated by a keyword"
    else:
        flag = "UNKNOW"          ##"Unable to understand the transition value. Check, something is horribly wrong here :"
    return flag

In [ ]:
def getvisit(cu, query_dict):
    res=query(cu, "SELECT urls.url, visits.visit_time, visits.visit_duration, visits.id, visits.from_visit, visits.transition, visits.segment_id, urls.id FROM visits INNER JOIN urls ON visits.url=urls.id;")
    parsed_res = []
    for item in res:
        urlhost = urlparse.urlparse(item[0]).netloc
        if item[7] in query_dict:
            queryterm = query_dict[item[7]]
        else:
            queryterm = ""
        parsed_res.append([urlhost, getTime(item[1]), getTime(item[2], "duration"), item[3], item[4], get_transition(item[5]), queryterm])
#         parsed_res.append([urlhost, item[1], item[2]])
    parsed_res.sort(key=lambda x:x[1])
    return parsed_res

In [11]:
visits = getvisit(cu, query_dict)
for v in visits[:30]:
    print "%s\t%s\t%s\t%s\t%s\t%s\t%s"%(v[1].strftime('%Y-%m-%d %H:%M:%S'), 
                            v[2].seconds,
                            v[3], 
                            v[4],
                            v[5],
                            v[6],
                            v[0])

NameError: name 'getvisit' is not defined

In [ ]:
def visitfilter(inlist, FILTER=None, DEL=False):
    res = []
    filter_in = ['dropbox', 'dictionary', 'facebook', 'mail', 'youtube', 'toasty', 'comic', 'dm5', 'mobile01']
    filter_out = ['account', 'ftp', 'itc']
    if FILTER == None:
        return inlist
    if FILTER == "Exclude":
        for item in inlist:
            if len([x for x in filter_out if x in item[0]])==0:
                res.append(item)
    if FILTER == "Include":
        for item in inlist:
            if len([x for x in filter_in if x in item[0]])>0:
                res.append(item)
    return res

In [ ]:
def merge_visit(visits):
    print "VISITS BEFORE MERGE:%d"%(len(visits))
    res = []
    uniq_url = None
    for visit in visits:
        if uniq_url != visit[0]:
            uniq_url = visit[0]
            res.append([uniq_url, visit[1], visit[1]+visit[2], visit[2]])
        else:
            try:
                res[-1][2] = visit[1]+visit[2]
                res[-1][3] += visit[2]
            except:
                print res[-1]
    print "VISITS BEFORE MERGE:%d"%(len(res))
    return res

In [ ]:

filtered_visits = visitfilter(visits, None) ##FILTER = Exclude, Include, or None
print "%s\t%-14s\t%s\t%-10s\t%-8s\t%s\t%s"%('date', 'time', 'duration', 'from', 'transition', 'to', 'url')
for r in filtered_visits[:10]:
    print "%s\t%-14s\t%s\t%-10s\t%-8s\t%s\t%s"%(r[1], r[2],  r[4], r[5], r[6], r[3], r[0])

In [12]:
merged_visits = merge_visit(filtered_visits)
print merged_visits[0]
for r in merged_visits:
#     print "%s\t%s\t%s\t%s"%(r[0], r[1].isoformat(), r[2].isoformat(), r[3].seconds)
    print "%s\t%s\t%s\t%s"%(r[1].strftime('%Y-%m-%d %H:%M:%S'), r[2].strftime('%Y-%m-%d %H:%M:%S'), r[3].seconds, r[0])

NameError: name 'merge_visit' is not defined

In [ ]:
def report(vlist, fname):
    fout=file(fname, 'w')
    fout.write('''
<script type="text/javascript" src="https://www.google.com/jsapi?autoload={'modules':[{'name':'visualization',
       'version':'1','packages':['timeline']}]}"></script>
<script type="text/javascript">

google.setOnLoadCallback(drawChart);
function drawChart() {

  var container = document.getElementById('example5.1');
  var chart = new google.visualization.Timeline(container);
  var dataTable = new google.visualization.DataTable();
  dataTable.addColumn({ type: 'string', id: 'Room' });
  dataTable.addColumn({ type: 'string', id: 'Name' });
  dataTable.addColumn({ type: 'date', id: 'Start' });
  dataTable.addColumn({ type: 'date', id: 'End' });
  dataTable.addRows([
    ''')
    for v in vlist:
        fout.write('["%s", "%s", new Date(0,0,0,%d,%d,%d), new Date(0,0,0,%d,%d,%d)],\n'%(v[1].strftime('%Y-%m-%d'), v[0], v[1].hour, v[1].minute, v[1].second, v[2].hour, v[2].minute, v[2].second))
    fout.write('''
        ]);
  var options = {
      colors:['#33ccff', '#ff66cc', '#339933','#ffcc33','#ff0000', '#333333', '#996600'],
    timeline: { rowLabelStyle: {fontName: 'Helvetica', fontSize: 24, color: '#603913' },
                barLabelStyle: { fontName: 'Garamond', fontSize: 14 } }
  };
  chart.draw(dataTable, options);
}
</script>
<div id="example5.1" style="width: 3900px; height: 1000px;"></div>
    ''')
    fout.close()


In [ ]:
def getPeriod(vlist, s, e):
    startTime = datetime.datetime.strptime(s, "%Y-%m-%d %H:%M:%S")
    endTime = datetime.datetime.strptime(e, "%Y-%m-%d %H:%M:%S")
    rlist=[]
    for v in vlist:
        if startTime < v[1] < endTime:
            if v[1].day != v[2].day:
                dayend = datetime.datetime.strptime(v[1].strftime('%Y-%m-%d')+" 23:59:59", "%Y-%m-%d %H:%M:%S")
                nextday = datetime.datetime.strptime(v[2].strftime('%Y-%m-%d')+" 00:00:00", "%Y-%m-%d %H:%M:%S")
#                 rlist.append([v[0], v[1], dayend, dayend-v[1]])
#                 rlist.append([v[0], nextday, v[2], v[2]-nextday])
            else:
                rlist.append(v)
    return rlist
#         if v[1] > v[2]:
#             print v[1].day, v[2].day, v[1].strftime('%Y-%m-%d %H:%M:%S'), v[2].strftime('%Y-%m-%d %H:%M:%S')
#         else:


In [ ]:
rlist = getPeriod(merged_visits, "2013-12-01 00:00:00", "2015-01-01 00:00:00")
for r in rlist:
    print r
report(rlist, "viz.html")

# Reference

In [ ]:
# Query a table
# def show_table(cu, name="%"):
#     sql="SELECT * FROM sqlite_master WHERE type='table' and name like '%s';"%(name)
#     return query(cu, sql)
# tb = show_table(cu) # return all table
# tb = show_table(cu, 'urls') # return table with name like urls